# Multidimensional Data and Singular Value Decomposition (SVD)
Author: Briony Yorke

In this workshop we will look at advanced methods to analyse multidimensional data. We will first revisit the idea of matrices and vectors and then we will look at *singular value decomposition* (SVD).


## Multidimensional Data

Scientific measurements often give us a set of numbers for each experiment.
For example:

- A spectrum with intensity at 1000 wavelengths

- A time-resolved signal measured at 200 timepoints

- An image with many pixels

- A sensor recording several channels at once

Each measurement is simply a list of numbers — a vector.

If you repeat the measurement many times (different samples or conditions), we can stack these vectors into a table:

Each row is one measurement

Each column is one variable (wavelength, timepoint, pixel, etc.)

For example:
If you take 10 spectra, each with 1000 wavelengths, your data becomes a table with 10 columns and 1000 rows:

A 10 × 1000 matrix.






### Organising Data with Matrices
A matrix is a convenient way to store several measurements together. We have already seen an example of matrix in the last workshop. We will be using atmospheric data again - this is multidimensional and includes measurements relating to local meteorology, such as temperature, pressure, wind speed, and wind direction, as well as measurements of the chemical composition of the atmosphere which includes concentrations of ozone, nitrogen oxides, hydrocarbons and the radicals OH and HO<sub>2</sub>.

Run the code cell below to import the packages you will use in this workshop and to import the data as a pandas dataframe.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.optimize import curve_fit
from scipy.cluster.hierarchy import linkage

file_url = 'http://homepages.see.leeds.ac.uk/~chmdst/big_data/Workshop4/clearflo_summer2012_15min_merge.csv'
df = pd.read_csv(file_url) # the header and delimiter are the defaults


Next you can check the determine and output size of the matrix (or dataframe) using the df.head() command in the code cell below:

In [ ]:
__.____()

To analyse the data in this table we will first remove any columns that do not contain numeric data (e.g. day_and_time).

We need to prepare the dataframe so that is only contains numeric data. To do this we will generate a new dataframe using the command ```df.select_dtypes``` and selecting columns that are numbers ```np.number```. 

Modify the cell below to generate a dataframe containing only numeric data and print the column headings. 

In [ ]:
# Select only numeric columns

numeric_df = ___.____(include=[___.___])

print("Numeric columns:",____.columns.tolist())


#### Pearson Correlation Coefficients

Our atmospheric data contains many variables, some of which correlate and others which are unrelated. We can quickly investigate correlations between pairs of variables by calculating the Pearson correlation coefficient.

The Pearson correlation coefficient is:

$\rho=\frac{cov(X/Y)}{\sigma_x\sigma_y}$

Where Cov(X,Y) is the covariance between two variables X and Y, the covariance is a measure of how two variables vary together. It gives you an idea of whether increases in one variable correspond to increases or decreases in another. $\sigma$ are the standard deviations of each varaible. 


- $\rho$ = 1 is a perfect positive correlation
- $\rho$ = -1 is a perfect negative correlation
- $\rho$ = 0 is no correlation

In the code cell below calculate the pairwise pearson correlation coefficients from the numeric dataframe ```numeric_df``` using the command ```corr_matrix=numeric_df.corr()```.


In [ ]:
# Compute correlation matrix
____ = _____.____()  # Pearson correlation

An effective way of visualising these correlations is to plot a **heatmap**. Heatmaps are a 2-dimensional data visualization technique that represent the magnitude of individual values within a matrix as a colour. Seaborn has an inbuilt heatmap function ```sns.heatmap```.

In the cell below use pandas to make a list of the column headers using the command ```df.columns.tolist()```. Then use  ```sns.heatmap``` to plot the correlation matrix ```corr_matrix``` using the colourmap (cmap) 'coolwarm', use the column headers as the y-ticklabels and x-ticklabels.

In [ ]:
column_headers= ____.____.____

plt.figure(figsize=(11, 9)) 
sns.heatmap(corr_matrix, cmap='___', yticklabels=___, xticklabels=___)

The diagonal line in the heat map shows that the variables perfectly correlate with themselves - this is exactly what we expect. The red regions in the heat map indicate where there is a strong colleration between variables, the pale colours indicate where there is no correlation (0.2 to -0.2).

The image below shows a section of the heat map highlighting a positive correlation between acetylene and ethene.

<img src="heatmap.png" height=500>

Ethene and acetylene have shared emission sources including:

- Vehicle exhaust

- Industrial combustion

- Biomass burning

- Natural gas leakage

- Petrochemical activity

So the positive correlation between C₂H₂ and C₂H₄ reflects shared emissions. The composition of the atmosphere is determined by complex chemistry, there are many different shared emission sources and chemical reactions that are dependent on multiple reagents, temperature and sun light. The chemical reactions are dependent on the emissions sources and one another. That means that the underlying chemistry that determines the atmospheric composition is multidimensional.

The correlation coefficient only shows us the relationship between pairs or variables, but understanding the relationship multiple variables is more complex.

Large scientific datasets often contain:

- Redundant information

- Noise

- Correlated variables

- A few “true” underlying features


Think of each variable as an axis, just like in 2 dimensions where we have an x-axis and y-axis, but now you have many axes:

O<sub>3</sub> axis, CO axis, NO axis,RO<sub>2</sub> axis etc ...

A single measurement is a point inside this huge multi-axis space.

A scatter plot usually only shows two dimensions x and y, we can visualise an additional dimension by using a colour map. Modify the code cell below to generate a scatter plot showing variation in O<sub>3</sub> / ppbv, CO /ppbv and PAN /pptv. This code uses the sns.scatterplot command from seaborn, seaborn is a package which can aid in data visualisation. It is especially useful for applying colour palettes (hue) to data, in this case we will use the palette called 'viridis'.

In [ ]:
sns.scatterplot(data=___, x='___', y='___', hue='___', palette='___', s=20)

Now try adding extra dimension by changing the size of the spots to correspond to temperature ```Temp / oC``` using the ```size=``` command:

In [ ]:
sns.scatterplot(data=___, x='___', y='___', hue='___', size='___', palette='_____', sizes=(5,50))

#### Checking our data
This doesn't look quite right - by plotting our data against temperature you can see there is some problem with the temperature column. The range of values is very low, we would never measure a temperature of -9000 <sup>o</sup>C. This shows us that data visualisation is very important not only to identify trends but also to find problems in our data that may not be obvious.

Let's quickly check our data. Modify the cell below to check the data format "dtype", the minimum "min" and maximum "max" values in the Temp / oC column and plot them. 

In [ ]:
#define the variable column and the column Temp / oC from the dataframe called numeric_df.
col = ___['___']

#print the datatype
print("dtype:", col.dtype)
#print the minimum value
print("min:", col.min())
#print the maximum value
print("___:", ___.___())

#plot the data
plt.figure()
plt.plot(___)
#set the title
____.____("Temperature check")
#show the plot
plt.show()

You can see that there are some outliers. We will remove the rows containing the outliers to simplify our data analysis. We will remove the rows from the data frame not the raw data - even the outliers may be important and should be preserved in the raw data.

First we need to determine which values are outliers, for this purpose can use the **z-value**. The z-value is a statistical measure that determines how many standard deviations a data point is from the mean:

z = (x - mean) / std

Run the cell below to use the stats.zscore command from the scipy stats package to calculate z-scores and then plot the z-scores.

In [ ]:
# Calculate the z-score for temperature
zscore = np.abs((col - col.mean()) / col.std())


# Plot z-scores as a scatter chart
plt.plot(zscore)
#set the title
plt.title("Zscores")
plt.xlabel("Data Set")
plt.ylabel("Z-Score")
#show the plot
plt.show()

We can now set a threshold for the z-score based off our graph, we can see that there are only a few outliers and the z-score of these are much greater than 2 standard deviations, count how many outliers are present and remove them from the dataframe.

Modify the code below to:

1. Define threshold as 2 standard deviations away from the mean (z-score = 2).
2. Define outliers in ```numeric_df``` as values with ```zscore > threshold```.
3. Print a list of outliers

In [ ]:
#define threshold
threshold = ___
#define outliers
outliers = ___[___ > ___]

# Print the outliers

print(____)


We can see there are 21 outliers to remove. To do this we will make a clean version of the data by making a new dataframe which only contains rows where the temperature is within 2 standard deviations of the mean.
>```python
> df_clean_temp = numeric_df[z < threshold]
>```

We will then check that 21 rows have been removed using the df.shape command to see the number of rows and columns.

In [ ]:
df_clean_temp = ___[___]   # keep points within ±2σ
df_clean_temp.shape

Now use the cell below to remove outliers in the solar power columns ```'Solar / Wm-2'``` you only need to repeat the z-score calculation, determine a sensible threshold and remove the columns where the z-score is more than 2 standard deviations away from the mean and generate a new dataframe called ```df_clean```.

In [ ]:
#define the variable column and the column Solar / Wm-2 from the dataframe called df_clean_temp.
col2 = ____-[_____]

# Calculate the z-score for temperature
zscore_solar = ____((____ - ____) / _____)


# Plot z-scores as a scatter chart
___.___(____)
#set the title
___.___("___")
#set the x-axis label
___.x___("___")
#set the y-axis label
plt.ylabel("___")
#show the plot
__.__()

#define threshold
threshold = ___
#define outliers
outliers = ____[____]

df_clean = ___[___< ___]   # keep points within the threshold standard deviation

#output the shape of the dataframe
–––.___


 

Now we have removed the outliers, let's retry making the plot where the temperature determines the size of the datapoints.

In [ ]:
sns.scatterplot(data=df_clean, x='O3 / ppbv', y='CO / ppbv', hue='PAN / pptv', size='Temp / oC', palette='viridis', sizes=(5,50))

Now we can get a better idea of trends in the data but this is still difficult to see and describe correlations between different variables. Since there are a huge number of variables in our data table (or matrix) the task of looking for correlations between different variables becomes even more difficult. Our plot is currently 4 dimensional, adding more dimensions would make it very difficult to interpret. Imagine if you wanted to find correlations between 1000s of variables - how can you do this quickly?

We can better understand our data using methods that delve into patterns or trends in our matrix.

#### Clusters
When you plot all measurements (even if you can’t visualize many dimensions), the points form some cloud-like shape or **cluster**. These can be elongated, flat, curved, thick in some directions and thin in others. In the plot above you may notice 'clustering' in the colours and it is possible to identify more structure in the data.

A “direction” is a line through the cloud or cluster. In principle component analysis and singular value decomposition this means a specific orientation in this multi-dimensional space. In 2D, a “direction” is just a line: ↗︎ or ↘︎ or horizontal or vertical. In 3D, it could be any vector inside the space. In multiple dimensions, it’s still a vector — just with a long list of coefficients.

You can think of this as rotating and moving the axes to find directions that show underlying relationships in the data.The directions are not necessarily aligned with one variable.

Imagine rotating a ruler inside your cloud of data, trying to find where the data spreads out the most.

The direction of maximum spread = 1st singular vector

The next-most-spread direction = 2nd singular vector

etc.

These directions are orthogonal (independent).

#### Why These Directions Matter
They reveal structure you don’t see by looking at raw variables.

## Singular Value Decomposition

SVD helps us find the fundamental **directions** in the data. These are called **singular vectors**.

SVD takes a matrix 𝐴 and rewrites it as:

$ A = U S V^T$

- Vᵀ contains the important directions in the data space. These explain how the data varies.

- U contains the weights (how much each direction contributes to each observation).

- S tells you the importance of each direction.

Large singular values = strong structure
Small singular values = noise

So SVD does the following factorisation: 

data → directions + strengths.

Taking a single matrix and breaking it down into the product of two or more simpler matrices is called **factorisation**.

Now let's look at a couple of example of factorisation, first we'll look at a very simple example.

Then we'll use a matrix that produces an image - this is similar to the skills you have learned in our image processing workshop. 

Run the cell below to generate and visualise a simple matrix.

In [ ]:
A = np.array([
    [1, 1, 0, 0],
    [1, 1, 0, 0],
    [0, 0, 2, 2],
    [0, 0, 2, 2]
], dtype=float)

plt.imshow(A, cmap='viridis')
plt.title("Original Matrix A")
plt.colorbar();



This matrix has two blocks:

- one block of 1’s

- one block of 2’s

It can be factorised as: 

𝐴 ≈ 𝐵𝐶

where

B is a 4×2 matrix and C is a 2×4 matrix.

This means we want to explain the 4×4 matrix using only two hidden patterns (the block of 1's and the block of 2's).

In the cell below we use a random seed to generate values for the elements of two matrices B and C. The rank tells us we are looking for 2 hidden patterns.

In [ ]:
np.random.seed(0)

rank = 2   # number of hidden patterns

B = np.random.rand(4, rank)
C = np.random.rand(rank, 4)

fig, ax = plt.subplots(1, 2, figsize=(8,4))

ax[0].imshow(B, cmap='viridis')
ax[0].set_title("Random B")


ax[1].imshow(C, cmap='viridis')
ax[1].set_title("Random C")


These random matrices are our first (random) guess but they do not satisfy the condition 𝐴 ≈ 𝐵𝐶, we can see this by multiplying matrices B and C using the @ symbol which performs matrix multiplication in NumPy. Run the cell below to reconstruct the matrix from the random seed.

In [ ]:
A_pred = B@C

fig, ax = plt.subplots(1, 2, figsize=(8,4))

ax[0].imshow(A, cmap='viridis')
ax[0].set_title("Original A")


ax[1].imshow(A_pred, cmap='viridis')
ax[1].set_title("Reconstructed A_pred")

plt.show()


A_pred does to look like our original matrix, we need to optimise the values in B and C so that we can reconstruct A.

We can do this by minimising the error :

error=∣∣A−BC∣∣

A simple way to do this is by using the gradient descent method.

In [ ]:
lr = 0.01  # learning rate this is the step size taken to update the parameters

for step in range(5000):
    # current reconstruction
    A_pred = B @ C

    # error matrix
    E = A - A_pred

    # update rules
    B += lr * (E @ C.T) # T is the transpose of the matrix - this flips the rows and columns to make sure B is the correct shape.
    C += lr * (B.T @ E)

fig, ax = plt.subplots(1, 2, figsize=(8,4))

ax[0].imshow(B, cmap='viridis')
ax[0].set_title("Best guess B")


ax[1].imshow(C, cmap='viridis')
ax[1].set_title("Best guess C")

# final reconstruction
A_pred = B @ C

# plot the matricees

fig, ax = plt.subplots(1, 2, figsize=(8,4))

ax[0].imshow(A, cmap='viridis')
ax[0].set_title("Original A")

ax[1].imshow(A_pred, cmap='viridis')
ax[1].set_title("Reconstructed A_pred")

plt.show()


Now we can see that we have predicted the underlying matrices and the two factors B and C have be optimised to reveal the hidden patterns in the matrix.

- one pattern corresponds to the “block of 1’s”

- one pattern corresponds to the “block of 2’s”


Now you can try this yourself:

Modify the cell below to generate a matrix called smiley.

In [ ]:
#define the matrix called smiley 

smiley = ___.___([
    [0,0,1,1,1,1,0,0],
    [0,1,0,0,0,0,1,0],
    [1,0,1,0,0,1,0,1],
    [1,0,0,0,0,0,0,1],
    [1,0,1,0,0,1,0,1],
    [1,0,0,1,1,0,0,1],
    [0,1,0,0,0,0,1,0],
    [0,0,1,1,1,1,0,0],
])

#plot the matrix as an image

___.___(___, ___="___")
#choose a suitable title
___.___("___")
___.___()


Generate 2 matrices $D$ and $F$ which have a size of 8 $\times$ 2 elements from a random seed and optimise them to minimise error=∣∣A−DF∣∣

In [ ]:
___.___.___(0)

rank = ___   # number of hidden patterns

D = ___.___.___(__, rank)
F = ___.___.___(rank, ___)

print(___)
print (___)


In [ ]:
lr = ___  # learning rate this is a hyperparameter - the step size taken to update the parameters

for ___ in ___(___):
    # current reconstruction
   ___ = ___ @ ___

    # error matrix
    E = ___ - ___

    # update rules
  ___ += lr * (E @ ___.T) # T is the transpose of the matrix - this flips the rows and columns to make sure B is the correct shape.
  ___ += lr * (___.T @ E)

# final reconstruction
___ = ___ @ ___

# plot the matricees smiley and smiley_pred using the viridis colour map

___, ___ = ___.___(___, ___, ___=(8,4))

ax[0].imshow(___, cmap='___')
ax[0].set_title("Original Smiley")

ax[1].imshow(___, cmap='___')
ax[1].set_title("Reconstructed smiley_pred")

___.___()

You can see that the optimisation has not completely worked. The reconstructed matrix looks different to the original - we have not been able to learn the hidden pattern.

Now let's compare this to the results that we can obtain using singular value decomposition.

Remember in SVD:

data → directions + strengths.

A = U S Vᵀ

In [ ]:
#Compute SVD
U, S, VT = np.linalg.svd(smiley)

print("U shape:", U.shape) 
print("S shape:", S.shape)
print("VT shape:", VT.shape)

#Reconstruct the matrix using only the first k components
def reconstruct(k):
    return (U[:, :k] @ np.diag(S[:k]) @ VT[:k, :])

plt.figure(figsize=(10,5))
for i,k in enumerate([1,2,4,8],1):
    plt.subplot(1,4,i)
    plt.imshow(reconstruct(k), cmap="viridis")
    plt.title(f"k = {k}")
    plt.axis("off")
plt.suptitle("Smiley Reconstructed With Increasing SVD Components")
plt.show()


Each SVD component is like a “layer” of the image. The first few components capture most of the structure.

SVD gives the optimal factorisation:

$ A = U S V^T $

If you want a rank-k approximation, you just keep the top k singular values:

$ A_k \approx U_k​S_kV_k^T$


This gives the best possible approximation to A, with the smallest reconstruction error, in a single computation.

### Why SVD is better than gradient descent

- No hyperparameters (no learning rate, no iterations).

- No risk of getting stuck at a bad solution.

- Fast and stable (thanks to 50 years of numerical linear algebra research).

- Error is provably minimal.

- It is mathematically the correct factorisation.

### Using SVD to analyse data

Next we'll use SVD to analyse a toy spectral dataset before analysing the atmospheric data.

Modify the cell below to:
1. Generate toy spectra of three different chemicals, A, B and C.

>```python
> x = np.linspace(200, 400, 300) '''First define the x-axis of the spectra. We will plot spectra with 300 data points between 200 - 400 nm.'''
>```

We will then generate spectra for the three different components as gaussian functions using the formula:

$Absorbance=Amplitude\times exp{-\frac{(\lambda-\lambda_{max})^2}{2\sigma^2}}$

Where: 
- $\lambda$ is the wavelength
- $\lambda_{max}$ is the wavelength at peak absorbance
- $\sigma$ is the width of the absorbance peak.

- Chemical A $\lambda_{max}$ = 230 nm and amplitdue 1
- Chemical B $\lambda_{max}$ = 290 nm and amplitude 0.9
- Chemical C $\lambda_{max}$ = 340 nm and amplitude 0.6

and $\sigma$ should be between 25-30 for each component.

Combine the three spectra into a 300 x 3 matrix using the vertical stack (vstack) command:

>```python
>  spectra = np.vstack([A, B, C]).T  '''# shape (300, 3) '''
>```

2. Define random weights to generate N = 200 random mixtures containing difference concentrations of the 3 chemicals. These weights alter the absorbance corresponding to an imagined change in concentration.

3. Add noise using ```np.random.randn``` ensuring the noise is added to every element in the mixed_abs matrix using ```(*mixed_abs.shape)```
4. Plot spectra of all the 200 random mixtures on one graph.

In [ ]:

np.random.seed(0)

# Create 3 chemical "spectra"

wl = np.linspace(180, 450, 300)

A = np.exp(-(wl-230)**2 / (2 * 25**2))        # species A
B = 0.9 * ___(-(___ - ___)**2 / (2 * ___**2))   # species B
C = ___ * ___(-(___ - ___)**___ / (___ * ___**___))  # species C

spectra = np.vstack([___, ___, ___]).T  # shape (300, 3)

 
#  Mix them into 200 samples

N = ___
weights = np.random.rand(N, 3)   # random abundances
mixed_abs = weights @ spectra.T           # resulting mixtures

# Add noise
mixed_abs += 0.2 * np.random.randn(___.___)

# Plot all spectra

# Create a magma palette with N colors

palette = sns.color_palette("magma", N)

for i in range(N):
    plt.plot(wl, mixed_abs[i], color=palette[i], alpha=0.6, linewidth=0.7)

plt.title("Spectra of 200 mixtures")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Absorbance")
plt.show()

These data are very noisy, it is very difficult to interpret the underlying spectra of the chemical species contributing to the mixture, it is also very difficult to determine the relative abundance of each species, we may not even know how many species are present in the mixtures. 

If we were to record these spectra we can use SVD to quickly:
1. Determine the number of chemical species in the mixtures.
2. Denoise the spectra.
3. Calculate the relative abundance of the components in each mixture.

Let's see what happens when we perform singular value decomposition. 

An important first step in SVD is to subtract the 'mean spectrum' from all of the indivdual spectra. This is called **centering**, we do this to remove offsets in the baseline and to prevent the mean spectrum from dominating the first singular vector. This means we focus on the differences in the spectra allowing us to determine the directions with the maximum variation.

To do this we make a new matrix called ```mixed_abs_centered``` which is equal to ```mixed_abs - np.mean(mixed_abs)``` we calculate the mean along each row of the matrix (spectrum) which is controlled by the ```axis=0``` command.

Modify the cell below to center and perform SVD.

In [ ]:

# Center the data

mixed_abs_centered = mixed_abs - np.mean(mixed_abs, axis=0)

# Perform SVD 

__, __, __ = ___.___.__(___, full_matrices=False)

#print the first 5 singular values

print("Singular values:", S[:5])

In this example:
- $U$ is the left singular vector, it contains information about how the abundance of each chemical in a sample.
- $S$ are the singular values. These measure how much variance each SVD component describes. Bigger singular value = more important spectral feature. They determine the scale of the component in the SVD reconstruction. A few components usally dominate the dataset and these have singular values which are much bigger than those representing noise.
- $V^T$  is the right singular vector, it is the shape basis vector over the 300 wavelengths. It contains information about the variance in the shape of the spectra.

We can see that the first 3 singular values dominate the spectra - suggesting there are 3 components. 

Next we can use a ***scree plot*** to show how important each SVD mode is.
The scree plot is a graph of the singular value based variance ($S^2$) against the singular value index (the component number)  a big drop after $S_3$ → indicates three strong dominating components. The scree plots helps identify the rank ($k$) of our spectra. 





In [ ]:
# Each singular value relates to variance as S^2
SVbased_variance = ___**___

# Make the Scree plot
plt.figure(figsize=(8,5))

# Scree plot
plt.plot(range(1, len(S)+1), SVbased_variance, marker='o', label='Variance explained')


plt.xlabel("Singular Value Index (Mode Number)")
plt.ylabel("Singular Value Based Variance ")
plt.title("___")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



The scree plot confirms that the first 3 components from the SVD are dominant. We could easily see this from our singular values but scree plots are very useful when there are many dominant vectors.

Now let's looks at the variance in the shape.

Modify the code below to plot the first 5 right singular vectors ($V^T$) and the spectra of the individual chemicals A, B and C. This code cotains a command to normalise the vector (divide by the maximum value) which ensures the x-axis represents wavelength:

>```python
>  sv_norm = sv / np.max(np.abs(sv))  # normalize for plotting
>```

In [ ]:
k = 5  # number of singular vectors to plot

# Plot first k singular vectors
plt.figure(figsize=(10,6))

for i in range(k):
    sv = VT[i]  # i-th singular vector
    sv_norm = ___ / ___.___(__.__(___))  # normalize for comparison
    plt.plot(wl, sv_norm, lw=2, label=f'SVD Vector {i+1}')

plt.title(f"Original Spectra and First {k} Singular Vectors")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Normalized Intensity")
plt.legend()
plt.show()

We have performed SVD on the mixed spectra. The number of components was not defined but we can clearly see that the first 3 right singular vectors ($V^T$) contain information about the varience in shape of the spectra, where as the fourth and fifth right singular vectors are just noise. Using SVD we have found that there are 3 components (chemicals)  in the mixtures.

Now modify the code cell below to plot the first 3 singular vectors and the spectra of the three chemicals A, B and C.

Make sure you normalise the singular vector as well as the original chemical spectra.

In [ ]:
plt.figure(figsize=(10,6))

# Plot original spectra
plt.plot(wl, A/np.max(A), 'k--', lw=1, label='Original A')
plt.plot(wl, B/np.max(B), 'k--', lw=1, label='Original B')
plt.plot(wl, C/np.max(C), 'k--', lw=1, label='Original C')

# Plot first k singular vectors
k = ___
for i in ___:
    ___ = ___[i]  # i-th singular vector
    ___ = ___ / ___.___(___.___(___))  # normalize for comparison
    ___.___(___, ___, ___=___, ___=f'___')

plt.title(f"First {k} Singular Vectors")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Normalized Intensity")
plt.legend()
plt.show()


You can see that the first 3 singular vectors gives us an indication of $ \lambda_{max}$ of the three chemicals. However, the SVD components do not perfectly match the original spectra of the three chemicals, they are oscillating patterns not Gaussians. 

This is because SVD rotates the mixture space into orthogonal axes.

### What do we mean by orthogonal?
Orthogonal means the dot product of two vectors is zero. For the first two right singular vectors from SVD, `VT[0]` and `VT[1]`

${VT[0]} \cdot \text{VT[1]} = \sum_{\lambda=1}^{M} \text{VT[0]}(\lambda) \, \text{VT[1]}(\lambda) = 0$

where, M, is the number of wavelengths (elements in each vector). This ensures that the singular vectors form perpendicular directions in wavelength space.

The original spectra A, B, C overlap → not orthogonal. SVD rotates the axes so that the first three singular vectors are orthogonal directions in wavelength space and finds the directions that best capture the variance. To remain orthogonal, these vectors often develop positive and negative lobes: positive regions “cancel out” negative regions when taking the dot product.

- The singular vectors capture variance, not the physical spectra.

- The original peak wavelength from singular vectors becomes more complex as the spectral overlap increases.

- But you can still determine the number of components and estimate abundances in the SVD space we can also reconstruct the spectra without noise.

Next we will see how SVD can be used to **denoise** our spectra.

First run the cell below to truncate to the first 3 components.

In [ ]:
k = 3
U_k = U[:, :k]        # (200,3)
S_k = np.diag(S[:k])  # (3,3)
VT_k = VT[:k, :]      # (3,300)


This is the rank-k approximation, we keep the top k singular vectors:

$ A_k \approx U_k​S_kV_k^T$

and reconstruct the original spectra from these three singular vectors by taking the dot product (mulitplication) of the left singular vector, the singular values and the right singular vetor.

Run the code cell below to calculate the reconstructed spectra.


In [ ]:
spectra_recon = (U_k @ S_k) @ VT_k  # shape (N, M)

To ensure the reconstructed spectra are on the correct scale we need to reverse the centering we performed before SVD. To do this we add the mean values back to the reconstructed values using the command:

>```python
> baseline = np.mean(mixed_abs, axis=0)
>spectra_recon_shifted = spectra_recon + baseline
>```

We must also remove negative values using the numpy clip to force the values to be non-negative (>=0)

- Any value below the minimum gets set to the minimum.

- Any value above the maximum gets set to the maximum.

- Values already inside the range stay unchanged.

Modify the code cell below to plot the first 5 original (mixed_abs) and reconstructed (spectra_recon) spectra. You should also calculate the root mean square error ($RMSE$) ```np.sqrt(np.mean(residuals**2))``` this was covered in workshop 4a.

In [ ]:
### enforce non-negativity (clipping)

baseline = np.mean(mixed_abs, axis=0)
spectra_recon_shifted = spectra_recon + baseline

spectra_recon_nn = np.clip(spectra_recon_shifted, a_min=0, a_max=None)

###compute residuals / metrics
residual = mixed_abs - spectra_recon
rmse_per_sample = ___(___.___(___**___, axis=1))
print("Overall RMSE:", ___(___.___(___**___)))

# plot original vs denoised for five spectra
nplot = ___
plt.figure(figsize=(15,20))
for ___ in ___(___):
    plt.subplot(nplot,1,i+1)
    ___.___(___, mixed_abs[___], color='___', label='original', alpha=0.8)
    plt.plot(___, spectra_recon_nn[___], color=___', label='reconstructed', lw=1)
    plt.ylabel('Intensity')
    if i==0:
        plt.legend()
plt.xlabel('___')
plt.ylabel('___')
plt.tight_layout()
___.___


We can see that the first three components capture the shape of the spectra and by reconstructing the spectra using only these components we can remove the noise. Denoising helps us to visualise the underlying spectra more clearly, we can use the peak heights and shapes from our denoised spectra in downstream analysis. 

Next we'll calculate the relative **abundance** of each chemical in the different mixtures (spectra) to do this we can use the $U$ matrix and the singular values vector $S$.

$K = U.S$ gives the abundances for each mixture in the SVD basis but for this calculation we need to perform the SVD on the uncentered matrices.

Run the cell below.

In [ ]:

# Scores = estimated abundances in SVD basis
k = 3
U_k = U[:, :k]        # (200,3)
S_k = np.diag(S[:k])  # (3,3)
VT_k = VT[:k, :]      # (3,300)

abundances_svd = U_k @ S_k  # shape (200,3)


If we plot the calculated 'SVD-based' abundancies against the weights that we calculated earlier we can see if they correlate.

### Rotation

Each row of $V^T_k$ is a singular vector across wavelengths. These vectors are orthogonal, so they usually don’t match the true spectra. We need to rotate the vectors using a rotaton matrix, R to put them in the correct orientation to determine abundancies.

$VT_k​R≈spectra$

We have the original spectra and so we can use a least-squares regression ```np.linalg.lstsq``` to find the correct rotation matrix.


We use also need to normalaise the weights using the command:

>```python
>weights_recovered_scaled = weights_recovered / np.max(weights_recovered, axis=0) * np.max(weights, axis=0)
> ```

¡¡¡ IMPORTANT !!! in the rotation command the underscores '_' do not represent a blank. You do not need to modify the cell below, just run it.




In [ ]:
#Rotate SVD components to match original spectra

rotation, _, _, _ = np.linalg.lstsq(VT_k.T, spectra, rcond=None)  # rotation: (3,3)

# Recover weights in original basis
weights_recovered = abundances_svd @ rotation  # (200,3)
weights_recovered_scaled = weights_recovered / np.max(weights_recovered, axis=0) 

#Compare recovered vs original

plt.figure(figsize=(8,5))
plt.scatter(weights[:,0], weights_recovered_scaled[:,0], alpha=0.6, label='A')
plt.scatter(weights[:,1], weights_recovered_scaled[:,1], alpha=0.6, label='B')
plt.scatter(weights[:,2], weights_recovered_scaled[:,2], alpha=0.6, label='C')
plt.xlabel("Original abundance")
plt.ylabel("Recovered abundance")
plt.title("Original vs Recovered Chemical Abundances")
plt.legend()
plt.show()

You can see that the weights calcualted from SVD correlate well with the weights used to generate our spectra! This is very useful when analysing a large number of mixtures for example we can use it to automatically detect how mixtures change during a chemical reaction!

## Analysing the atmospheric dataset using SVD

We will use SVD to analyse the atmospheric dataset, each column in the dataframe contains information about different chemical species. Now we are investigating the most important directions in our data across these variables, rather than across wavelength space that we used in the previous example. 

The directions with the strongest variance in this space can tell us about relationships between the different variables e.g. are the concentrations of 

To perform SVD on the atmospheric data we will first convert the clean dataframe ```df_clean``` to a matrix. Run the code cell below to convert and also print out the size of the matrix ```M.shape```.

In [ ]:
M = df_clean.to_numpy()

print("Data matrix shape:", M.shape)

In [ ]:

#Center the data (subtract mean)

M_centered = ___ - ___.___(___, axis=0)

# Perform SVD

___,___, ___ = ___.___.___(___, full_matrices=False)

print("U shape:", ___.___)
print("S shape:", ___.___)
print("VT shape:", ___.___)



#### What do the single values tell us?

Single values tell us about the number of dominant components.

Use the code cell below to print all of the 100 singular values and generate a scree plot to evaluate the number of dominant components in the dataset.

In [ ]:
print("Singular values:", ___[:___])

# Each singular value relates to variance as S^2
SVbased_variance = ___**___

# Make the Scree plot
plt.figure(figsize=(8,5))

# Scree plot
___.___(___(___, ___(S)+1), ___, marker='o', label='Variance explained')


plt.xlabel("Singular Value Index (Mode Number)")
plt.ylabel("Singular Value Based Variance ")
plt.title("SVD Scree Plot")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

The first mode strongly dominates, this means that most of the day-to-day variability across ALL 55 variables is controlled by ONE underlying atmospheric process. The process corresponding to the largest singular value is typically a strong, coherent process that affects many species at once. In atmospheric observations, a typical dominant process is photochemical activity. 

However, there are losts of large singular values, they are less clear on the scree plot due to the size of the first value but they are still important. We can plot the scree plot on a logarithmic y-axis ```plt.yscale("log")``` to get more information about the other components. Modify the code cell below to generate the log scree plot.

In [ ]:
# log scree plot
___.___(___(___, ___(S)+1), ___, marker='o', label='Variance explained')

plt.xlabel("Singular Value Index (Mode Number)")
plt.ylabel("Singular Value Based Variance ")
plt.title("SVD Scree Plot")
plt.yscale("___")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

From the log scree plot you can see that most of the components are large and that most of them contribute to the variance. The last 5 singular vectors are small. If the data had unimportant variables these would be flat. 

#### What do the shapes of the right singular vectors tell us
Plot the right singular vectors of the first 3 components.

In [ ]:
num_components=___

plt.figure(figsize=(14,6))

for ___ in range(___):
    ___.___(___[___], marker='o', label=f"Component {___}")
    
plt.xticks(ticks=np.arange(len(column_headers)), labels=column_headers, rotation=90)
plt.xlabel("Variables")
plt.ylabel("Right singular vector value")
plt.title("Right Singular Vectors of Atmospheric Chemistry Data")
plt.legend()
plt.tight_layout()
plt.show()




You can see that the directions with the maximum variance are dominated by $HO_2$, $RO_2$ and $CH_3O_2$

These are peroxy radicals

- $HO_2$: hydroperoxyl radical

- $RO_2$: generic organic peroxy radical

- $CH_3O_2$: methyl peroxy radical

They are key intermediates in atmospheric oxidation of VOCs (volatile organic compounds) and CO. During the day they are generated rapidly by photochemical reactions via OH-initiated oxidation of VOCs. They also react rapidly with NO. This rapid production/removal leads to large fluctuations in their concentrations on short timescales. 

Small changes in NO, sunlight, or VOC emissions cause big swings in radical concentrations. This makes these species dominate the first principal components (maximum variance directions). Molecules like CO, CH₄, or even many VOCs (such as alkenes, alkanes and aromatics) vary more slowly. This means their variance is smaller, so they contribute less to the directions of maximum variance in SVD.

In the code cell below plot the 5th right singular vector:

In [ ]:
plt.figure(figsize=(14,6))

plt.plot(___[___], marker='o', label=f"Component {___}")
 #set xticks   
___.___(___=___.___(___(____)),____=____, rotation=90)
#set x-label as 'Variables'
___.___("___")
#set y-label as 'Right singular vector value'
___.___("___")
#set plot title as Right Singular Vectors of Atmospheric Chemistry Data
___.___("___")
#add legend
___.___()
#set tight layout 
___.___()
#show plot
___.___()

We can see that different variables contribute to each right singular vector ($V^T$), these singular vectors give us an indication of which variables change with one another.

- Variables with large positive or negative values in the same right singular vector tend to co-vary across the dataset:

- If two variables have similar signs and magnitudes → they increase/decrease together along that component.

- If opposite signs → they vary in opposite directions.

### What do the left singular vectors tell us?

The left singular vectors ($U$) show us the temporal patterns of the SVD component that is how that component changes over time. Use the code cell below to plot the first 5 components and see how they vary with time.

In [ ]:

#Plot first five left singular vectors

plt.figure(figsize=(10, 6))
for k in range(components):
    plt.plot(___[:, ___], label=f'U[{___}]')
plt.xlabel("Time index")
plt.ylabel("Amplitude")
plt.title("Left singular vectors (first few modes)")
plt.legend()
plt.tight_layout()
plt.show()


We can see from this plot that these components oscillate in time. We can explain this by thinking about photochemistry, during the day photochemical processes will dominate, however, at night other processes will be taking place.

We can caluclate the correlation coefficient between each left singular vector ($U$) and each of the variables and display these on a heat map to identify which of the variables most strongly oscillate in time.

Run the code cell below:

In [ ]:

# Set Parameters 
n_components = 50  # number of left singular vectors to analyse


# Compute correlation matrix ----
# rows = SVD components, columns = variables
corr_matrix = np.zeros((n_components, len(cols)))

for k in range(n_components):
    for j, col in enumerate(cols):
        corr_matrix[k, j] = np.corrcoef(U[:, k], df_clean[col].values)[0, 1]

corr_df = pd.DataFrame(corr_matrix, columns=cols, index=[f'U[{k}]' for k in range(n_components)])

# Plot heatmap ----
plt.figure(figsize=(20, 6))  # wider figure
sns.heatmap(corr_df, cmap='coolwarm', center=0, annot=False, cbar_kws={'label': 'Correlation'})
plt.xlabel("Chemical species")
plt.ylabel("Left singular vectors (U modes)")
plt.title("Correlation of left singular vectors with chemical species")
plt.xticks(ticks=np.arange(len(cols)) + 0.5, labels=cols, rotation=90)  # force all labels
plt.tight_layout()
plt.show()

This shows us that the radical species correlate most strongly with the first left singular vector $U[0]$, these are generated by photochemical processes so this makes perfect sense due to the daily oscillations in solar power. We can also that the majority of other species are oscillating in time, this could be due to any number of factors including reacting slowly with products of radical reactions, pollutant emissions, wind etc.
